### Урок 4. Сверточные нейронные сети

#### Домашнее задание

##### 1. обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10

##### 2. оценить рост точности при увеличении ширины сети (больше фильтров)

##### 3. оценить рост точности при увеличении глубины сети (больше слоев)

#####  *сравнить с точностью полносвязной сети для этой выборки
##### Рекомендации по оформлению кода:
исправляя код из ноутбука, указывайте, пожалуйста, где были сделаны изменения, чтобы было легче ориентироваться в коде, к тому же это повысит скорость проверки работы.

In [1]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

/Users/annvorosh/miniforge3/envs/mlp/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


Python Platform: macOS-13.2.1-arm64-arm-64bit
Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.9.16 (main, Mar  8 2023, 04:29:24) 
[Clang 14.0.6 ]
Pandas 1.5.3
Scikit-Learn 1.2.2
SciPy 1.10.0
GPU is NOT AVAILABLE


In [2]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [3]:
from keras.datasets import cifar10

num_classes = 10


# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# нормирование данных
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [ ]:

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Функция, которая создает модель
def create_model():
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(64, (2, 2), padding='same',  #32
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))        # 32                                 
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))   # (2,2)
    model.add(Dropout(0.25))


    # -------------------------------------------------------------------------- добавим слои
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # -------------------------------------------------------------------------- добавим слои

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.summary()
    
    # инициализация RMSprop optimizer
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# создаем экземпляр модели для использования с Grid Search
model = KerasClassifier(build_fn=create_model, verbose=0)

# определяем параметры, которые будут оптимизироваться
batch_size = [16] # 32 , 64, 128
epochs = [30] # , 20
param_grid = dict(batch_size=batch_size, epochs=epochs)

# запускаем Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3) #
grid_result = grid.fit(x_train, y_train)

# выводим результаты
print("Лучший результат: %f с использованием %s" % (grid_result.best_score_, grid_result.best_params_))
# тоже самое можно провернуть и с другими параметрами (кол-во слоев сети, размер фильтра и тд)

<ipython-input-4-4574d2a132bb>:54: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Лучший результат: 0.714820 с использованием {'batch_size': 32, 'epochs': 20} # Dense 512

Лучший результат: 0.715160 с использованием {'batch_size': 32, 'epochs': 20} # model.add(Conv2D(32,(2, 2),input... изменили размер ядра сверточного слоя с 3х3 на 2х2

Лучший результат: 0.701000 с использованием {'batch_size': 32, 'epochs': 20}  # Dense 128  больше фильтров 64

Лучший результат: 0.718980 с использованием {'batch_size': 16, 'epochs': 30}  # добавили слои Dense(512) 2х2+3x3 

#### Лучший результат: 0.737040 с использованием {'batch_size': 16, 'epochs': 30} # добавили слои Dense(512  фильтров 64

Лучший результат: 0.723480 с использованием {'batch_size': 16, 'epochs': 30} 

Лучший результат: 0.660520 с использованием {'batch_size': 32, 'epochs': 20} # Dense 64

Лучший результат: 0.682060 с использованием {'batch_size': 32, 'epochs': 20} # Dense 128

Лучший результат: 0.696000 с использованием {'batch_size': 32, 'epochs': 30} # Dense 64 

Лучший результат: 0.652720 с использованием {'batch_size': 16, 'epochs': 30}  # добавили слои Dense(64) 2х2
      
Лучший результат: 0.703680 с использованием {'batch_size': 16, 'epochs': 30}  #  Dense(64)

Лучший результат: 0.680620 с использованием {'batch_size': 16, 'epochs': 30}  #  Dense(64) добавили слои ????   
       
### Выводы
При увеличении ширины сети (больше фильтров) точность сети возрастает.

При увеличении глубины сети (больше слоев) точность сети возрастает.

Так же точность сети возрастает при увеничении количества нейронов в Dense слое, при увеличении количества эпох и при снижении размера батча.